# Fine Tuning ng Open AI Models

Ang notebook na ito ay batay sa kasalukuyang gabay na makikita sa [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) na dokumentasyon mula sa Open AI.

Pinapahusay ng fine-tuning ang performance ng foundation models para sa iyong aplikasyon sa pamamagitan ng muling pagsasanay gamit ang karagdagang datos at konteksto na may kaugnayan sa partikular na use case o sitwasyon. Tandaan na ang mga teknik tulad ng _few shot learning_ at _retrieval augmented generation_ ay nagbibigay-daan para mapabuti ang default na prompt gamit ang mga kaugnay na datos upang mapataas ang kalidad. Gayunpaman, limitado ang mga pamamaraang ito ng maximum token window size ng napiling foundation model.

Sa fine-tuning, muling sinasanay mismo ang model gamit ang kinakailangang datos (na nagbibigay-daan para magamit ang mas maraming halimbawa kaysa sa kasya sa max token window) - at nagde-deploy ng isang _custom_ na bersyon ng model na hindi na kailangang bigyan ng mga halimbawa tuwing inference. Hindi lang nito pinapabuti ang bisa ng ating prompt design (mas malaya tayong magamit ang token window para sa ibang bagay) kundi maaari ring makatipid sa gastos (dahil nababawasan ang bilang ng tokens na kailangang ipadala sa model tuwing inference).

Ang fine tuning ay may 4 na hakbang:
1. Ihanda ang training data at i-upload ito.
1. Patakbuhin ang training job para makakuha ng fine-tuned na model.
1. Suriin ang fine-tuned na model at ulitin ang proseso para sa kalidad.
1. I-deploy ang fine-tuned na model para sa inference kapag kontento na.

Tandaan na hindi lahat ng foundation models ay sumusuporta sa fine-tuning - [tingnan ang dokumentasyon ng OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) para sa pinakabagong impormasyon. Maaari ka ring mag-fine-tune ng model na dati nang na-fine-tune. Sa tutorial na ito, gagamitin natin ang `gpt-35-turbo` bilang target foundation model para sa fine-tuning.

---


### Hakbang 1.1: Ihanda ang Iyong Dataset

Gagawa tayo ng chatbot na tutulong sa iyo na maintindihan ang periodic table ng mga elemento sa pamamagitan ng pagsagot ng mga tanong tungkol sa isang elemento gamit ang isang limerick. Sa _tutorial_ na ito, gagawa lang tayo ng dataset para sanayin ang modelo gamit ang ilang halimbawa ng mga sagot na nagpapakita ng inaasahang format ng data. Sa totoong paggamit, kailangan mong gumawa ng dataset na may mas maraming halimbawa. Maaari ka ring gumamit ng open dataset (para sa iyong application domain) kung meron, at i-reformat ito para magamit sa fine-tuning.

Dahil nakatuon tayo sa `gpt-35-turbo` at naghahanap ng isang beses na sagot (chat completion), maaari tayong gumawa ng mga halimbawa gamit ang [inirerekomendang format na ito](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) na sumusunod sa mga kinakailangan ng OpenAI chat completion. Kung inaasahan mong magkakaroon ng usapan na may maraming palitan ng mensahe, gagamitin mo ang [multi-turn example format](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) na may kasamang `weight` parameter para tukuyin kung aling mga mensahe ang dapat (o hindi dapat) gamitin sa fine-tuning process.

Gagamitin natin ang mas simpleng single-turn format para sa tutorial na ito. Ang data ay nasa [jsonl format](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) na may 1 record bawat linya, bawat isa ay isang JSON-formatted object. Ang snippet sa ibaba ay nagpapakita ng 2 record bilang halimbawa - tingnan ang [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) para sa buong sample set (10 halimbawa) na gagamitin natin para sa fine-training tutorial. **Tandaan:** Ang bawat record _ay dapat_ nakasulat sa isang linya lang (hindi hinati-hati sa maraming linya gaya ng karaniwan sa formatted JSON file)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

Sa totoong paggamit, kailangan mo ng mas malaking set ng mga halimbawa para sa mas magagandang resulta - ang magiging kapalit ay ang kalidad ng mga sagot at ang oras/gastos ng fine-tuning. Maliit lang ang set na ginagamit natin dito para mabilis nating matapos ang fine-tuning at maipakita ang proseso. Tingnan ang [OpenAI Cookbook na halimbawa na ito](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) para sa mas kumplikadong fine-tuning tutorial.


### Hakbang 1.2 I-upload ang Iyong Dataset

I-upload ang data gamit ang Files API [gaya ng ipinaliwanag dito](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Tandaan na bago mo mapatakbo ang code na ito, kailangan mo munang gawin ang mga sumusunod na hakbang:
 - Na-install ang `openai` Python package (siguraduhing ang bersyon ay >=0.28.0 para sa mga pinakabagong features)
 - Na-set ang `OPENAI_API_KEY` environment variable gamit ang iyong OpenAI API key
Para sa karagdagang impormasyon, tingnan ang [Setup guide](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) na ibinigay para sa kursong ito.

Ngayon, patakbuhin ang code para gumawa ng file na ia-upload mula sa iyong lokal na JSONL file.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### Hakbang 2.1: Gumawa ng Fine-tuning na trabaho gamit ang SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### Hakbang 2.2: Suriin ang Status ng job

Narito ang ilang bagay na maaari mong gawin gamit ang `client.fine_tuning.jobs` API:
- `client.fine_tuning.jobs.list(limit=<n>)` - Ilista ang huling n fine-tuning jobs
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Kunin ang detalye ng isang partikular na fine-tuning job
- `client.fine_tuning.jobs.cancel(<job_id>)` - Kanselahin ang isang fine-tuning job
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Ilista hanggang n events mula sa job
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Ang unang hakbang sa proseso ay _beripikahin ang training file_ para matiyak na tama ang format ng data.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### Hakbang 2.3: Subaybayan ang mga kaganapan para makita ang progreso


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Hakbang 2.4: Tingnan ang status sa OpenAI Dashboard


Maaari mo ring tingnan ang status sa pamamagitan ng pagbisita sa website ng OpenAI at pag-explore sa seksyong _Fine-tuning_ ng platform. Ipapakita nito sa iyo ang status ng kasalukuyang trabaho, at maaari mo ring subaybayan ang kasaysayan ng mga naunang pagpapatakbo ng trabaho. Sa screenshot na ito, makikita mo na nabigo ang naunang pagpapatakbo, at nagtagumpay ang pangalawang run. Para sa konteksto, nangyari ito nang gumamit ang unang run ng JSON file na may maling format ng mga record – nang maitama ito, matagumpay na natapos ang pangalawang run at naging available na ang modelong magamit.


Maaari mo ring makita ang mga mensahe ng status at mga sukatan sa pamamagitan ng pag-scroll paibaba sa visual dashboard gaya ng ipinapakita:

| Mga Mensahe | Mga Sukatan |
|:---|:---|
| ![Mga Mensahe](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.tl.png) |  ![Mga Sukatan](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.tl.png)|


### Hakbang 3.1: Kunin ang ID at Subukan ang Fine-Tuned na Modelo sa Code


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### Hakbang 3.2: I-load at Subukan ang Fine-Tuned na Model sa Playground

Pwede mo nang subukan ang fine-tuned na model sa dalawang paraan. Una, maaari mong bisitahin ang Playground at gamitin ang Models drop-down para piliin ang bago mong fine-tuned na model mula sa mga opsyon na nakalista. Ang isa pang paraan ay gamitin ang "Playground" na opsyon na makikita sa Fine-tuning panel (tingnan ang screenshot sa itaas) na magbubukas ng _comparative_ na view kung saan makikita mo ang foundation at fine-tuned na model na magkatabi para sa mabilisang pagsusuri.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.tl.png)

Ilagay lang ang system context na ginamit mo sa training data at ibigay ang iyong test question. Mapapansin mong parehong na-update ang magkabilang panig gamit ang parehong context at tanong. Patakbuhin ang comparison at makikita mo ang pagkakaiba ng outputs ng dalawa. _Pansinin kung paano sinusunod ng fine-tuned na model ang format ng sagot na ibinigay mo sa mga halimbawa, habang ang foundation model ay sumusunod lang sa system prompt_.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.tl.png)

Mapapansin mo rin na ipinapakita ng comparison ang bilang ng tokens para sa bawat model, pati na rin ang oras na ginugol para sa inference. **Ang partikular na halimbawa na ito ay simple lang at layuning ipakita ang proseso, ngunit hindi talaga sumasalamin sa totoong dataset o sitwasyon.** Maaaring mapansin mong pareho ang bilang ng tokens ng parehong sample (magkapareho ang system context at user prompt) pero mas matagal ang inference ng fine-tuned na model (custom model).

Sa totoong mga sitwasyon, hindi ka gagamit ng ganitong simpleng halimbawa, kundi magfi-fine-tune gamit ang totoong data (halimbawa, product catalog para sa customer service) kung saan mas halata ang kalidad ng sagot. Sa _ganung_ konteksto, para makuha ang kaparehong kalidad ng sagot gamit ang foundation model, kakailanganin ng mas masusing prompt engineering na magdudulot ng mas mataas na token usage at posibleng mas matagal na processing time para sa inference. _Para subukan ito, tingnan ang mga fine-tuning na halimbawa sa OpenAI Cookbook bilang panimula._



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama’t nagsusumikap kami para sa kawastuhan, pakatandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi eksaktong pagsasalin. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring lumitaw mula sa paggamit ng pagsasaling ito.
